In [1]:
import sys 
sys.dont_write_bytecode = True

import tensorflow as tf
import numpy as np
import argparse
import scipy.sparse as sp

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

sess = tf.InteractiveSession()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [13]:
dense_w = {
    "w_ip1": tf.Variable(tf.truncated_normal([784, 1024], stddev=0.1), name="w_ip1"),
    "b_ip1": tf.Variable(tf.constant(0.1, shape=[1024]), name="b_ip1"),
    "w_ip2": tf.Variable(tf.truncated_normal([1024, 1024], stddev=0.1), name="w_ip2"),
    "b_ip2": tf.Variable(tf.constant(0.1, shape=[1024]), name="b_ip2"),
    "w_ip3": tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1), name="w_ip3"),
    "b_ip3": tf.Variable(tf.constant(0.1, shape=[10]), name="b_ip3"),
}

In [14]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
keep_prob = tf.placeholder("float")

In [15]:
def dense_dnn_model(weights):
    x_image = tf.reshape(x, [-1, 28*28])
    h_ip1 = tf.nn.relu(tf.matmul(x_image, weights["w_ip1"]) + weights["b_ip1"])
    h_ip1_drop = tf.nn.dropout(h_ip1, keep_prob)
    h_ip2 = tf.nn.relu(tf.matmul(h_ip1_drop, weights["w_ip2"]) + weights["b_ip2"])
    h_ip2_drop = tf.nn.dropout(h_ip2, keep_prob)
    y_conv = tf.nn.softmax(tf.matmul(h_ip2_drop, weights["w_ip3"]) + weights["b_ip3"])
    return y_conv

In [32]:
def test(y_infer, message="None."):
    correct_prediction = tf.equal(tf.argmax(y_infer, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    result = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
    
    print(message+" %g\n" %result)
    return result

In [16]:
y_conv = dense_dnn_model(dense_w)

In [17]:
saver = tf.train.Saver()

In [18]:
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())

In [35]:
for i in range(50000):
    batch = mnist.train.next_batch(50)
    if i%500 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

score = test(y_conv, message="training test accuracy")
score

step 0, training accuracy 0.92
step 500, training accuracy 0.92
step 1000, training accuracy 0.92
step 1500, training accuracy 0.9
step 2000, training accuracy 0.9
step 2500, training accuracy 0.88
step 3000, training accuracy 0.94
step 3500, training accuracy 0.88
step 4000, training accuracy 0.94
step 4500, training accuracy 0.94
step 5000, training accuracy 0.94
step 5500, training accuracy 0.88
step 6000, training accuracy 0.92
step 6500, training accuracy 0.96
step 7000, training accuracy 0.96
step 7500, training accuracy 0.94
step 8000, training accuracy 0.92
step 8500, training accuracy 0.92
step 9000, training accuracy 0.84
step 9500, training accuracy 0.96
step 10000, training accuracy 0.92
step 10500, training accuracy 0.92
step 11000, training accuracy 0.92
step 11500, training accuracy 0.9
step 12000, training accuracy 0.94
step 12500, training accuracy 0.92
step 13000, training accuracy 0.94
step 13500, training accuracy 0.92
step 14000, training accuracy 0.9
step 14500, t

0.93410009

In [36]:
saver.save(sess, "./model_ckpt_dense", write_meta_graph=False)

'./model_ckpt_dense'